In [0]:
def assign_data(data,centers):
  # n is the number of data points
  n = len(data)
  # d is the dimensionality of the data points
  d = len(data[0])
  # k is the number of clusters
  k = len(centers)
  # first, subtract the set of centers from each data point
  res = np.reshape(data,(1,n,d))-np.reshape(centers,(k,1,d))
  # sum the squared differences
  res2 = np.add.reduce(res**2,2)
  # assign each data point to its closest center
  centerids = np.apply_along_axis(np.argmin,0,res2)
  # While we're here, make a note of the loss
  loss = sum(np.apply_along_axis(np.min,0,res2))
  return(centerids, loss)

In [0]:
def compute_means(data, centerids, k):
  # n is number of data points
  n = len(data)
  # d is dimensionality of the data points
  d = len(data[0])
 
  # Zero out the centers
  centers = np.zeros(shape=(k,d))
 
  # loop over the clusters
  for i in range(k):
    # Gather the data points assigned to cluster i
    cols = np.array([data[j] for j in range(n) if centerids[j] == i])
    # Average to get mean for that cluster
    if len(cols) == 0: 
      centers[i] = data[random.randint(0,n-1)]
    else:
      centers[i] = cols.mean(0)
  return(centers)

In [0]:
def kmeans(data, k):
  n = len(data)
  d = len(data[0])
  # grab the centers from random points
  centers = data[[random.randint(0,n-1) for i in range(k)]]
  oldloss = 0
  loss = 1
  while oldloss != loss:
    oldloss = loss
    centerids, loss = assign_data(data,centers)
    centers = compute_means(data, centerids, k)
  return(centers, loss)

In [4]:
from sklearn.datasets import fetch_openml
data = fetch_openml(name='mnist_784')

import numpy as np
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.1)
X_train, X_test, y_train, y_test = train_test_split(X_test, y_test, test_size=0.33)
len(X_train)

4690

In [5]:
from scipy import stats
import math
from functools import reduce
import random

#for nlabeled in range(20,len(X_train),10):
nlabeled = 20
if True:
  print(nlabeled)
  ans = []
  k = 10 # 2 # 5 # 20
  if True:
    bestcenters, bestloss = kmeans(X_train, k)
    for rep in range(9):
      centers, loss = kmeans(X_train, k)
      if loss < bestloss: bestcenters, bestloss = centers, loss
    # How do we test the clustering that was discovered?
    # Assign testing points to clusters
    test_centerids, loss = assign_data(X_test, bestcenters)

    # Use the labeled examples to label the clusters
    train_centerids, loss = assign_data(X_train[:nlabeled], bestcenters)
    #print(train_centerids)
    #print(y_train[:nlabeled])
    labs = y_train[:nlabeled]

#    clust_labs = np.zeros(shape=(k))
    clust_labs = np.repeat(labs[0],k)
    for i in range(k):
      mode = stats.mode(labs[train_centerids == i]).mode
      if len(mode) > 0: clust_labs[i] = mode[0]

# print(clust_labs)
    ans = ans + [(k,sum(clust_labs[test_centerids] == y_test)/len(y_test))]
#    plt.plot(X_test[clust_labs[test_centerids] == 0,0],X_test[clust_labs[test_centerids] == 0,1],'o',color='r')
#    plt.plot(X_test[clust_labs[test_centerids] == 1,0],X_test[clust_labs[test_centerids] == 1,1],'o',color='b')
#    plt.show()

#  print(ans)
  print(reduce((lambda x, y: x if x[1] > y[1] else y), ans))
  labids, loss = assign_data(X_test, X_train[:nlabeled])
  print(nlabeled, sum(y_train[labids] == y_test)/len(y_test))

20
(10, 0.4683982683982684)
20 0.5229437229437229


In [6]:
from keras.preprocessing.image import array_to_img

train_centerids, loss = assign_data(X_train, bestcenters)
test_centerids, loss = assign_data(X_test, bestcenters)

clust_labs = np.repeat(labs[0],k)
for i in range(len(bestcenters)):
  display(array_to_img(np.reshape(bestcenters[i],(28,28,1)), scale=False))
  clust_labs[i] = y_train[train_centerids == i][0]
  print(clust_labs[i])
#  mode = stats.mode(y_train[train_centerids == i]).mode
#  print(mode[0])
#  if len(mode) > 0: clust_labs[i] = mode[0]

sum(clust_labs[test_centerids] == y_test)/len(y_test)

Using TensorFlow backend.


3


6


7


1


...

0


0


2


2


0.48095238095238096

In [10]:
from scipy import stats
import math
from functools import reduce

# ACTIVE LEARNING VERSION

#for nlabeled in range(20,len(X_train),10):
nlabeled = 10
if True:
  ans = []
  k = 50 # 10 # 2 # 5 # 20
  if True:
#  for k in range(10,200,50):
    bestcenters, bestloss = kmeans(X_train, k)
    for rep in range(9):
      centers, loss = kmeans(X_train, k)
      if loss < bestloss: bestcenters, bestloss = centers, loss
    # How do we test the clustering that was discovered?
    # Assign testing points to clusters
    test_centerids, loss = assign_data(X_test, bestcenters)

    # Let's label one example in each category
    train_centerids, loss = assign_data(X_train, bestcenters)

    clust_labs = np.repeat(labs[0],k)
    for i in range(len(bestcenters)):
      clust_labs[i] = y_train[train_centerids == i][0]

    # semi-supervised clustering
    print(k,sum(clust_labs[test_centerids] == y_test)/len(y_test))

    # nearest neighbors
    labids, loss = assign_data(X_test, X_train[:k])
    print(k, sum(y_train[labids] == y_test)/len(y_test))

50 0.7597402597402597
50 0.6095238095238096
